In [3]:
import torch
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import lr_scheduler
from torchvision import transforms

import numpy as np
import pandas as pd
from PIL import Image
import time

import librosa
import librosa.display

import matplotlib.pyplot as plt

import deepdish as dd

import utils
from trainer.UNet import UNet
from trainer.utils import FMADataset

In [4]:
X = dd.io.load('data/fma_preprocessed/X_3.h5')
Y = dd.io.load('data/fma_preprocessed/Y_3.h5')

In [5]:
# standardize images
mean = np.mean(X['training'])
std = np.std(X['training'])

for k in X.keys():
    X[k] = (X[k] - mean) / std

In [22]:
test_acc = 0.

test_set = FMADataset(X, Y, mode='test')
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

model_paths = [
    'results/11/model_intermediate0.4892_B42_LR0.0002607264726285735.pth',
    'results/10/model_intermediate0.4888_B42_LR0.0004042345079351265.pth',
    'results/3/model_B42_LR0.0002996498451642352.pth',
    'results/3/model_intermediate0.4817_B42_LR0.0002996498451642352.pth',
    'results/22/model_SOneCycleLR_B42_LR0.009414192900389376.pth',
    'results/27/model_SOneCycleLR_B41_LR0.010132984105149187.pth',
    'results/30/model_intermediate0.4779_B42_LR0.00030461574329755707.pth',
    'results/30/model_intermediate0.4704_B42_LR0.00030461574329755707.pth'
]

# Load model
model = UNet()
model.load_state_dict(torch.load(model_paths[1], map_location=torch.device('cpu')))
model.eval()

for xs, ys in test_loader:
    probs = model(xs)
    ys = torch.max(ys, 1)[1]
    pred = probs.cpu().argmax(1)
    test_acc += (pred == ys).sum()
    
test_acc /= len(test_loader.dataset)

print(test_acc)

tensor(0.4104)
